In [1]:
import matplotlib.pyplot as plt
from random import sample
import networkx as nx
import pandas as pd
import numpy as np

In [2]:
from os import listdir
from os.path import isfile, join

In [3]:
root_dir = os.getcwd()

In [4]:
file_set = set()

for dir_, _, files in os.walk(root_dir):
    for file_name in files:
        rel_dir = os.path.relpath(dir_, root_dir)
        rel_file = os.path.join(rel_dir, file_name)
        file_set.add(rel_file)

In [5]:
filtered_file_set = [x for x in file_set if "RPD_output\\graph\\" in x]

In [6]:
len(filtered_file_set)

108

In [7]:
dataset_files = []
for file in filtered_file_set:
    network_data = pd.read_excel(file, sheet_name = ['Sheet1'], index_col=None)
    dataset_files.extend(network_data['Sheet1'].values.tolist())
df_export = pd.DataFrame(dataset_files)


In [8]:
df_export.rename(columns={1:'disc', 2:'vacancy_id', 3: 'oop_name', 4: 'speciality', 5: 'cos_simularity'}, inplace=True)
df_export.drop(columns=df_export.columns[0], axis=1, inplace=True)

In [9]:
df_export.to_excel('text.xlsx')

In [10]:
network_data = df_export

In [11]:
connections_data = network_data[network_data['cos_simularity'] >= 0.5]
edge_cols = ['oop_name', 'speciality', 'cos_simularity']

In [12]:
df_disc_unique = network_data[['disc', 'oop_name']].drop_duplicates()
df_disc_unique.rename(columns={"disc": "node_label", "oop_name": "attribute"}, inplace=True)

In [13]:
df_vac_unique = network_data[['vacancy_id', 'speciality']].drop_duplicates()
df_vac_unique.rename(columns={"vacancy_id": "node_label", "speciality": "attribute"}, inplace=True)

In [14]:
elements_data = df_disc_unique.append(df_vac_unique, ignore_index=True)
elements_data.to_excel('text1.xlsx')

C:\Users\seismopro\AppData\Local\Temp\ipykernel_11764\855689358.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elements_data = df_disc_unique.append(df_vac_unique, ignore_index=True)


In [15]:
graph = nx.convert_matrix.from_pandas_edgelist(connections_data,
                                               source = 'disc',
                                               target = 'vacancy_id',
                                               edge_attr = edge_cols)

In [16]:
node_dict = elements_data.set_index('node_label').to_dict(orient = 'index')


In [17]:
node_dict['85915191']

{'attribute': 'Финансовый менеджер'}

In [18]:
nx.set_node_attributes(graph, node_dict)
%matplotlib inline

In [19]:
graph.add_nodes_from(elements_data['node_label'])

In [20]:
nx.number_of_isolates(graph)

3485

In [21]:
nx.write_graphml(graph, "RPD_output\graph1.graphml")

In [23]:
'''fig = plt.figure(figsize = (14, 9))
#colors = np.linspace(0, 1, len(graph.nodes))
layout = nx.spring_layout(graph, k=0.70, iterations=60)
graph.remove_nodes_from(list(nx.isolates(graph)))

nx.draw(graph, 
        node_size  = 30,
        #node_color = colors,
        pos = layout, 
        node_color = node_dict,
        edge_color = 'g')
fig.set_facecolor('k')'''

"fig = plt.figure(figsize = (14, 9))\n#colors = np.linspace(0, 1, len(graph.nodes))\nlayout = nx.spring_layout(graph, k=0.70, iterations=60)\ngraph.remove_nodes_from(list(nx.isolates(graph)))\n\nnx.draw(graph, \n        node_size  = 30,\n        #node_color = colors,\n        pos = layout, \n        node_color = node_dict,\n        edge_color = 'g')\nfig.set_facecolor('k')"